# Event-based Simulation with SimPy

In [1]:
import simpy as sp
import numpy as np
import scipy as scp
import scipy.stats
from typing import List, Tuple, Dict, Union

# alternatively to using np.random's distributions,
#   you can use a distribution directly from scipy.stats:
# import scipy.stats

## SimPy Basics

SimPy has a few basic concepts to let you perform event-based simulation:
1. **Environment:** think of this as "the world" in which your simulation occurs
2. **Event:** an event that occurs in the environment -- it can be a happening, from time ticking to an action occurring
3. **Process:** described by Python generator functions -- they create events and `yield` (yeet) them out to the world
4. **Timeout:** a special time-based event, that describes how long it takes for an event to occur (assumed units)
5. **Resource:** an object representing a resource that can be acquired or released by Processes (gas station with limited fuel pumps)
6. **Container:** an object representing a resource that can be produced or consumed (like electrical power or apples)
7. **Store:** an object representing a store that can replenish or sell its items (dynamic quantities)

One thing that is key about event-based simulation is that, much like embedded systems, all processes are **interruptible** by priority, meaning that processes can be at different priorities (just like real life).

If you need to yield multiple events (actions) at a time, use the `&` operator.  If you need to yield at least one event (whichever event is closest), use the `|` operator.

In [2]:
def clock(env: sp.Environment, name: str, tick: float):
    """
    generator: some clock that ticks at "tick" seconds
    
    :param env: the universe (SimPy Environment)
    :param name: the name of the clock
    :param tick: how often this clock will speak
    """
    while True:
        print(f"clock {name}, current time: {env.now:.2f}")
        yield env.timeout(tick)

In [3]:
test_env = sp.Environment()
clock_slow = clock(test_env, "slow", 2.0)
clock_fast = clock(test_env, "fast", 0.5)
clock_ultra = clock(test_env, "ultra", 0.1)

In [4]:
clock_ultra

<generator object clock at 0x7f8892b2e580>

In [5]:
test_env.process(clock_ultra)
test_env.process(clock_slow)
test_env.process(clock_fast)

<Process(clock) object at 0x7f8892b2fd00>

In [6]:
test_env.run(until=5.)

clock ultra, current time: 0.00
clock slow, current time: 0.00
clock fast, current time: 0.00
clock ultra, current time: 0.10
clock ultra, current time: 0.20
clock ultra, current time: 0.30
clock ultra, current time: 0.40
clock fast, current time: 0.50
clock ultra, current time: 0.50
clock ultra, current time: 0.60
clock ultra, current time: 0.70
clock ultra, current time: 0.80
clock ultra, current time: 0.90
clock ultra, current time: 1.00
clock fast, current time: 1.00
clock ultra, current time: 1.10
clock ultra, current time: 1.20
clock ultra, current time: 1.30
clock ultra, current time: 1.40
clock fast, current time: 1.50
clock ultra, current time: 1.50
clock ultra, current time: 1.60
clock ultra, current time: 1.70
clock ultra, current time: 1.80
clock ultra, current time: 1.90
clock slow, current time: 2.00
clock fast, current time: 2.00
clock ultra, current time: 2.00
clock ultra, current time: 2.10
clock ultra, current time: 2.20
clock ultra, current time: 2.30
clock ultra, cu

### DMV customer example

Just to show everything in context, I'll modify the [Bank example](https://simpy.readthedocs.io/en/latest/examples/bank_renege.html) to the DMV:

In [12]:
RANDOM_SEED = 42
NEW_CUSTOMERS = 60  # Total number of customers
INTERVAL_CUSTOMERS = 1.0  # Generate new customers roughly every x minutes
MIN_PATIENCE = 10  # Min. customer patience
MAX_PATIENCE = 40  # Max. customer patience


def source(
    env: sp.Environment, 
    number: int,  # number of customers to spawn
    interval: float,  # some time interval into the exponential dist.
    counter: sp.Resource,  # the DMV counter
    the_rng: np.random.Generator
):
    """Source ("the door") generates customers randomly"""
    for i in range(number):
        customer_generator = customer(env, f"Customer {i:2}", counter, 12.0, the_rng)
        env.process(customer_generator)
        spawn_time = the_rng.exponential(interval)
        yield env.timeout(spawn_time)


def customer(
    env: sp.Environment, 
    name: str, 
    counter: sp.Resource, 
    time_request_takes: float, 
    the_rng
):
    """Customer arrives (spawned by "the door"), waits to get served and leaves after they're served."""
    arrive = env.now
    print(f"at time {arrive:7.2f}, {name} arrived")

    with counter.request() as req:
        patience = the_rng.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)

        wait = env.now - arrive

        if req in results:
            # We got to the counter
            print('at time %7.2f, %s waited %6.2f' % (env.now, name, wait))

            tib = the_rng.exponential(time_request_takes)
            yield env.timeout(tib)
            print('at time %7.2f, %s finished their transactions' % (env.now, name))

        else:
            # We reneged
            print('at time %7.2f, %s left the line after waiting %6.2f minutes' % (env.now, name, wait))


# Setup and start the simulation
print('SIMULATION: DMV in all of its glory (time units are minutes)')
the_rng = np.random.default_rng(RANDOM_SEED)
env = sp.Environment()

# Start processes and run
counter = sp.Resource(env, capacity=2)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter, the_rng))
env.run()

SIMULATION: DMV in all of its glory (time units are minutes)
at time    0.00, Customer  0 arrived
at time    0.00, Customer  0 waited   0.00
at time    2.40, Customer  1 arrived
at time    2.40, Customer  1 waited   0.00
at time    2.68, Customer  2 arrived
at time    4.09, Customer  3 arrived
at time    4.17, Customer  4 arrived
at time    4.24, Customer  5 arrived
at time    5.98, Customer  6 arrived
at time    7.21, Customer  7 arrived
at time    7.30, Customer  8 arrived
at time    8.20, Customer  9 arrived
at time    9.45, Customer 10 arrived
at time   11.28, Customer 11 arrived
at time   11.94, Customer 12 arrived
at time   12.39, Customer 13 arrived
at time   12.57, Customer 14 arrived
at time   12.96, Customer 15 arrived
at time   13.67, Customer 16 arrived
at time   14.13, Customer 17 arrived
at time   14.48, Customer 18 arrived
at time   15.35, Customer 19 arrived
at time   16.69, Customer 20 arrived
at time   17.77, Customer 21 arrived
at time   18.91, Customer 22 arrived
at

### Sample SimPy problem

Let's perform a simulation of four electric cars (EV, electric vehicles) trying to drive to and use a battery charging station. A car will perform three actions:
1. Driving to the EV charging station (accept the driving time as an input parameter).
2. Request/acquire a charging spot.
3. Charge the battery (accept the time it takes to charge the battery as an input parameter).

Define the charging station as only having two spots, so we have a likely chance of at least one car waiting for charging to complete.
Let's use a uniform distribution between 2 and 6 miles to model the distance each car has to drive in order to reach the EV charging station. Let's also assume that the cars can drive 35 miles per hour to reach the charging station.  Let's also use a uniform distribution to model the number of minutes that each car has to charge, from 30 minutes to 90 minutes.

LPT: Don't forget the random seed!

In [13]:
RANDOM_SEED = 7
CHARGING_SPOTS = 2

In [14]:
def create_electric_cars(
    env: sp.Environment,
    charging_station: sp.Resource,
    the_rng: np.random.Generator,
    number_of_cars: int = 4,
    distance_min_mi: float = 2.,
    distance_max_mi: float = 6.,
    driving_speed_mph: float = 35.,
    charging_min_minutes: float = 30.,
    charging_max_minutes: float = 90.,
    charging_spots: int = 2
) -> None:
    for car_id in range(number_of_cars):
        distance_mi = the_rng.uniform(
            low=distance_min_mi,
            high=distance_max_mi)
        charging_minutes = scp.stats.uniform.rvs(
            loc=charging_min_minutes,
            scale=(charging_max_minutes - charging_min_minutes),
            random_state=the_rng)
        driving_minutes = 60. * distance_mi / driving_speed_mph
        print(f"T {env.now}, Car {car_id} with driving time {driving_minutes:.2f} and charging time {charging_minutes:.2f} to be added.")
        env.process(electric_car_behavior(env, charging_station, car_id, driving_minutes, charging_minutes))
    print(f"T {env.now}, all cars added to the environment.")
    
def electric_car_behavior(
    env: sp.Environment,
    charging_station: sp.Resource,
    name: str,
    driving_minutes: float,
    charging_minutes: float
):
    print(f"T {env.now:.2f}, Car {name} with driving time {driving_minutes:.2f} and charging time {charging_minutes:.2f} spawned.")
    yield env.timeout(driving_minutes)  # 1. driving to the station
    
    with charging_station.request() as charging_spot:  # 2a. request a charging spot
        wait_start_minutes = env.now
        print(f"T {env.now:.2f}, Car {name} is waiting for a charging spot.")
        yield charging_spot  # 2b. tell the simulator to come back when a spot exists
        
        wait_end_minutes = env.now
        wait_duration_minutes = wait_end_minutes - wait_start_minutes
        print(f"T {env.now:.2f}, Car {name} waited for {wait_duration_minutes:.2f} mins for a charging spot.")
        
        yield env.timeout(charging_minutes)  # 3. wait until battery is charged
    
    print(f"T {env.now:.2f}, Car {name} finished charging and left.")

In [15]:
the_rng = np.random.default_rng(RANDOM_SEED)
the_env = sp.Environment()
the_charging_station = sp.Resource(the_env, capacity=CHARGING_SPOTS)

create_electric_cars(the_env, the_charging_station, the_rng)
the_env.run()

T 0, Car 0 with driving time 7.71 and charging time 83.83 to be added.
T 0, Car 1 with driving time 8.75 and charging time 43.51 to be added.
T 0, Car 2 with driving time 5.49 and charging time 82.41 to be added.
T 0, Car 3 with driving time 3.46 and charging time 79.27 to be added.
T 0, all cars added to the environment.
T 0.00, Car 0 with driving time 7.71 and charging time 83.83 spawned.
T 0.00, Car 1 with driving time 8.75 and charging time 43.51 spawned.
T 0.00, Car 2 with driving time 5.49 and charging time 82.41 spawned.
T 0.00, Car 3 with driving time 3.46 and charging time 79.27 spawned.
T 3.46, Car 3 is waiting for a charging spot.
T 3.46, Car 3 waited for 0.00 mins for a charging spot.
T 5.49, Car 2 is waiting for a charging spot.
T 5.49, Car 2 waited for 0.00 mins for a charging spot.
T 7.71, Car 0 is waiting for a charging spot.
T 8.75, Car 1 is waiting for a charging spot.
T 82.74, Car 3 finished charging and left.
T 82.74, Car 0 waited for 75.02 mins for a charging spot.

## Handling priorities and interrupts

### Using PriorityResource
Let's bump the EV charging simulation up now, to 10 cars, and pretend that a couple of the cars have made reservations online, so they can jump the queue ([`simpy.PriorityResource`](https://simpy.readthedocs.io/en/latest/topical_guides/resources.html#priorityresource)).

In [16]:
def create_electric_cars_with_priorities(
    env: sp.Environment,
    charging_station: sp.Resource,
    the_rng: np.random.Generator,
    number_of_cars: int = 4,
    distance_min_mi: float = 2.,
    distance_max_mi: float = 6.,
    driving_speed_mph: float = 35.,
    charging_min_minutes: float = 30.,
    charging_max_minutes: float = 90.,
    charging_spots: int = 2
) -> None:
    for car_id in range(number_of_cars):
        distance_mi = the_rng.uniform(
            low=distance_min_mi,
            high=distance_max_mi)
        charging_minutes = scp.stats.uniform.rvs(
            loc=charging_min_minutes,
            scale=(charging_max_minutes - charging_min_minutes),
            random_state=the_rng)
        driving_minutes = 60. * distance_mi / driving_speed_mph
        charging_station_priority = scp.stats.hypergeom.rvs(
            M=10,
            n=3,
            N=6,
            random_state=the_rng)
        env.process(
            electric_car_behavior_with_priorities(
                env, charging_station, car_id, driving_minutes, charging_minutes, charging_station_priority))
    print(f"T {env.now}, all cars added to the environment.")
    
def electric_car_behavior_with_priorities(
    env: sp.Environment,
    charging_station: sp.Resource,
    name: str,
    driving_minutes: float,
    charging_minutes: float,
    charging_station_priority: int
):
    print(f"T {env.now:.2f}, Car {name} with priority {charging_station_priority} driving time {driving_minutes:.2f} and charging time {charging_minutes:.2f} spawned.")
    yield env.timeout(driving_minutes)  # 1. driving to the station
    
    with charging_station.request(priority=charging_station_priority) as charging_spot:  # 2a. request a charging spot
        wait_start_minutes = env.now
        print(f"T {env.now:.2f}, Car {name} (Prio {charging_station_priority}) is waiting for a charging spot.")
        yield charging_spot  # 2b. tell the simulator to come back when a spot exists
        
        wait_end_minutes = env.now
        wait_duration_minutes = wait_end_minutes - wait_start_minutes
        print(f"T {env.now:.2f}, Car {name} (Prio {charging_station_priority}) waited for {wait_duration_minutes:.2f} mins for a charging spot.")
        
        yield env.timeout(charging_minutes)  # 3. wait until battery is charged
    
    print(f"T {env.now:.2f}, Car {name} (Prio {charging_station_priority}) finished charging and left.")

In [17]:
the_rng = np.random.default_rng(RANDOM_SEED)
the_env = sp.Environment()
the_charging_station = sp.PriorityResource(the_env, capacity=CHARGING_SPOTS)

create_electric_cars_with_priorities(
    the_env, the_charging_station, the_rng,
    number_of_cars=10
)
the_env.run()

T 0, all cars added to the environment.
T 0.00, Car 0 with priority 2 driving time 7.71 and charging time 83.83 spawned.
T 0.00, Car 1 with priority 1 driving time 9.42 and charging time 30.32 spawned.
T 0.00, Car 2 with priority 1 driving time 5.34 and charging time 45.29 spawned.
T 0.00, Car 3 with priority 2 driving time 10.25 and charging time 77.56 spawned.
T 0.00, Car 4 with priority 3 driving time 4.90 and charging time 39.61 spawned.
T 0.00, Car 5 with priority 2 driving time 9.72 and charging time 67.75 spawned.
T 0.00, Car 6 with priority 1 driving time 3.51 and charging time 41.54 spawned.
T 0.00, Car 7 with priority 2 driving time 4.49 and charging time 46.06 spawned.
T 0.00, Car 8 with priority 3 driving time 7.82 and charging time 74.51 spawned.
T 0.00, Car 9 with priority 1 driving time 9.40 and charging time 51.68 spawned.
T 3.51, Car 6 (Prio 1) is waiting for a charging spot.
T 3.51, Car 6 (Prio 1) waited for 0.00 mins for a charging spot.
T 4.49, Car 7 (Prio 2) is wai

**Notice:** You can see that in the above output has cars with higher priority (lower number) that have "jumped the queue" over cars with lower priority (higher number).

### Using a time-based interrupt

Interrupts in a fair way, to indicate when a driver might have just gotten a text message and needs to leave and be somewhere instead.  We can simulate this by adding a "scheduled event" timeout in addition to the charging time timeout using the "|" (or) operator:

In [23]:
def create_cars_with_prio_leavetime(
    env: sp.Environment,
    charging_station: sp.Resource,
    the_rng: np.random.Generator,
    number_of_cars: int = 4,
    distance_min_mi: float = 2.,
    distance_max_mi: float = 6.,
    driving_speed_mph: float = 35.,
    charging_min_minutes: float = 30.,
    charging_max_minutes: float = 90.,
    charging_spots: int = 2
) -> None:
    for car_id in range(number_of_cars):
        charging_time = the_rng.uniform(charging_min_minutes, charging_max_minutes)
        driving_distance = the_rng.uniform(distance_min_mi, distance_max_mi)
        driving_time = 60 * driving_distance / driving_speed_mph
        # use beta distribution below so most cars have
        #   lower priority (priority 1) than VIPs (priority 0)
        priority = np.round(the_rng.beta(5, 3)).astype(int)
        leave_time = the_rng.uniform(10., charging_max_minutes)
        env.process(
            electric_car_with_prio_leavetime(
                env, charging_station, car_id,
                driving_time, charging_time, priority, leave_time))
    print(f"At {env.now}, all cars have been added to the environment.")


def electric_car_with_prio_leavetime(
    env: sp.Environment, charging_station: sp.PriorityResource,
    name: str, driving_time: float, charging_time: float,
    charging_station_priority: int, leave_time: float
):
    print(f"At {env.now:.2f}, Car {name} with t_drive {driving_time:.2f} and t_charge {charging_time:.2f} has spawned.")
    print(f"At {env.now:.2f}, Car {name} is driving to the EV charging station.")
    yield env.timeout(driving_time)  # 1. driving to the station
    
    with charging_station.request(priority=charging_station_priority) as charging_spot:
        wait_start = env.now
        print(f"At {env.now:.2f}, Car {name} with priority {charging_station_priority} "
              f"is waiting for an EV charging spot.")
        yield charging_spot  # 2b. wait for an available EV charging spot
        
        wait_completed = env.now
        wait_duration = wait_completed - wait_start
        print(f"At {env.now:.2f}, Car {name} with priority {charging_station_priority} "
              f"has acquired an EV charging spot after waiting {wait_duration:.2f}.")
        # either the charging or the leave timeout will trigger 
        #   first, use timeout_event to store which
        timeout_event = yield env.timeout(charging_time, value="charging_timeout") | env.timeout(leave_time, value="leave_timeout")
        
    print(f"At {env.now:.2f}, Car {name} has finished charging and has left due to {list(timeout_event.values())}.")

In [24]:
the_rng = np.random.default_rng(RANDOM_SEED)
the_env = sp.Environment()
the_charging_station = sp.PriorityResource(the_env, capacity=CHARGING_SPOTS)
create_cars_with_prio_leavetime(the_env, the_charging_station, the_rng)
the_env.run()

At 0, all cars have been added to the environment.
At 0.00, Car 0 with t_drive 9.58 and t_charge 67.51 has spawned.
At 0.00, Car 0 is driving to the EV charging station.
At 0.00, Car 1 with t_drive 8.89 and t_charge 79.27 has spawned.
At 0.00, Car 1 is driving to the EV charging station.
At 0.00, Car 2 with t_drive 7.22 and t_charge 60.27 has spawned.
At 0.00, Car 2 is driving to the EV charging station.
At 0.00, Car 3 with t_drive 3.67 and t_charge 32.64 has spawned.
At 0.00, Car 3 is driving to the EV charging station.
At 3.67, Car 3 with priority 1 is waiting for an EV charging spot.
At 3.67, Car 3 with priority 1 has acquired an EV charging spot after waiting 0.00.
At 7.22, Car 2 with priority 1 is waiting for an EV charging spot.
At 7.22, Car 2 with priority 1 has acquired an EV charging spot after waiting 0.00.
At 8.89, Car 1 with priority 1 is waiting for an EV charging spot.
At 9.58, Car 0 with priority 1 is waiting for an EV charging spot.
At 36.31, Car 3 has finished charging

**Notice:** Some of the cars above waited the whole charging time, others have just left because they need to go somewhere else.

### Using PreemptiveResource 

We could even say someone has a VIP card that can kick someone off of a pump ([`simpy.PreemptiveResource`](https://simpy.readthedocs.io/en/latest/topical_guides/resources.html#preemptiveresource)), though that would probably not be a fair system at all.  However, you can imagine there are simulation cases where you would want [preemption](https://en.wikipedia.org/wiki/Preemption_(computing)), such as in an embedded system that requires fast response times, networks that require resources to be shutdown for maintenance, etc. 

To catch when an interrupt occurs, we have to use a [try-except statement](https://docs.python.org/3/tutorial/errors.html#handling-exceptions).  We also have to use a while loop to re-enter the line.  Let's see it in action: 

In [29]:
def create_cars_with_prio_leave_vip(
    env: sp.Environment,
    charging_station: sp.Resource,
    the_rng: np.random.Generator,
    number_of_cars: int = 4,
    distance_min_mi: float = 2.,
    distance_max_mi: float = 6.,
    driving_speed_mph: float = 35.,
    charging_min_minutes: float = 30.,
    charging_max_minutes: float = 90.,
    charging_spots: int = 2
) -> None:
    for car_id in range(number_of_cars):
        charging_time = the_rng.uniform(charging_min_minutes, charging_max_minutes)
        driving_distance = the_rng.uniform(distance_min_mi, distance_max_mi)
        driving_time = 60 * driving_distance / driving_speed_mph
        # use beta distribution below so most cars have
        #   lower priority (priority 1) than VIPs (priority 0)
        priority = np.round(the_rng.beta(5, 3)).astype(int)
        leave_time = the_rng.uniform(10., charging_max_minutes)
        env.process(electric_car_with_prio_leave_vip(
            env, charging_station, car_id, driving_time,
            charging_time, priority, leave_time))
    print(f"At {env.now}, all cars have been added to the environment.")


def electric_car_with_prio_leave_vip(env: sp.Environment, charging_station: sp.PriorityResource,
                                     name: str, driving_time: float, charging_time: float,
                                     charging_station_priority: int, leave_time: float):
    # setup placeholders for data
    charge_left = charging_time  # store the charging time needed separately
    leave_left = leave_time  # store the leave time separately
    timeout_event = None  # store the timeout event for later use
    charging_timeout = None  # store the charging timeout for later use
    leave_timeout = None  # store the leave timeout for later use
    reenter_line = True  # state variable to keep track of whether to re-enter the line
    
    print(f"At {env.now:.2f}, Car {name} with t_drive {driving_time:.2f}, "
          f"t_charge {charging_time:.2f}, and t_leave {leave_time:.2f} has spawned.")
    print(f"At {env.now:.2f}, Car {name} is driving to the EV charging station.")
    yield env.timeout(driving_time)
    
    # while loop so we can re-insert this car into the queue automatically
    while reenter_line:
        with charging_station.request(priority=charging_station_priority) as charging_spot:
            wait_start = env.now
            leave_timeout = env.timeout(leave_left, value="leave_timeout")
            print(f"At {env.now:.2f}, Car {name} with priority {charging_station_priority} "
                  f"is waiting for an EV charging spot.")
            timeout_event = yield charging_spot | leave_timeout
            
            if leave_timeout in timeout_event:
                print(f"At {env.now:.2f}, Car {name} gave up waiting in line...")
                return

            wait_completed = env.now
            wait_duration = wait_completed - wait_start
            print(f"At {env.now:.2f}, Car {name} with priority {charging_station_priority} "
                  f"has acquired an EV charging spot after waiting {wait_duration:.2f}.")
            
            # We have to use a try-except-finally to handle the interrupt:
            try:
                charge_start = env.now
                charging_timeout = env.timeout(charge_left, value="charging_timeout")
                timeout_event = yield charging_timeout | leave_timeout
                reenter_line = False  # leave the line if we finished
            except sp.Interrupt as the_interrupt:
                interrupter = the_interrupt.cause.by
                print(f"At {env.now:.2f}, Car {name} was interrupted by {interrupter}. "
                      f"Getting back in line and waiting additional 10...")
                leave_left += 10.
            finally:
                # this lets us execute the computation in either case
                charge_duration = env.now - charge_start
                charge_left -= charge_duration
                leave_left -= charge_duration
    
    if charging_timeout in timeout_event:
        print(f"At {env.now:.2f}, Car {name} has finished charging and left.")
    elif leave_timeout in timeout_event:
        print(f"At {env.now:.2f}, Car {name} has to go somewhere and left with "
              f"{charge_left:.2f} charge required.")

In [30]:
the_rng = np.random.default_rng(RANDOM_SEED)
the_env = sp.Environment()
the_charging_station = sp.PreemptiveResource(the_env, capacity=CHARGING_SPOTS)
create_cars_with_prio_leave_vip(the_env, the_charging_station, the_rng)
the_env.run()


At 0, all cars have been added to the environment.
At 0.00, Car 0 with t_drive 9.58, t_charge 67.51, and t_leave 10.42 has spawned.
At 0.00, Car 0 is driving to the EV charging station.
At 0.00, Car 1 with t_drive 8.89, t_charge 79.27, and t_leave 45.61 has spawned.
At 0.00, Car 1 is driving to the EV charging station.
At 0.00, Car 2 with t_drive 7.22, t_charge 60.27, and t_leave 59.00 has spawned.
At 0.00, Car 2 is driving to the EV charging station.
At 0.00, Car 3 with t_drive 3.67, t_charge 32.64, and t_leave 51.13 has spawned.
At 0.00, Car 3 is driving to the EV charging station.
At 3.67, Car 3 with priority 1 is waiting for an EV charging spot.
At 3.67, Car 3 with priority 1 has acquired an EV charging spot after waiting 0.00.
At 7.22, Car 2 with priority 1 is waiting for an EV charging spot.
At 7.22, Car 2 with priority 1 has acquired an EV charging spot after waiting 0.00.
At 8.89, Car 1 with priority 1 is waiting for an EV charging spot.
At 9.58, Car 0 with priority 1 is waitin

**Notice:** We can capture if a car gives up in line, but if we extended the leave timeout then you can also see interrupted cars wait it out.

## Monitoring your SimPy simulation

There are three ways to go about monitoring a simulation, in order from easy to difficult:
1. Implement your own monitoring harness (basically, add input/output data structures (such as lists) for monitoring into your process functions and classes)
2. Patching (creating a wrapper) for Simpy `Events` ([see this link](https://simpy.readthedocs.io/en/latest/topical_guides/monitoring.html#event-tracing))
3. Patching (creating a wrapper) for SimPy `Resources` ([see this link](https://simpy.readthedocs.io/en/latest/topical_guides/monitoring.html#resource-usage))

So, we can go ahead and use a dictionary and some lists to basically make our monitoring harness.  If we want to be fancier and make it available for analysis immediately after, we can use a pandas DataFrame after.  Either way, we must think about how we want to store our data (the columns of the table, the type of information, the granularity).  Sometimes this is called a _data schema_.

So, let's go ahead and log the following information: Event time, Car ID, Car Priority, Drive Time, Available Time Left, Charge Time Left, Event

Let's see how to setup our previous code so that we can finalize it into a `pandas.DataFrame` so we can easily get analyze it:

In [ ]:
def create_cars_with_prio_leave_vip(env: sp.Environment, charging_station: sp.Resource,
                                    rng: np.random.Generator,
                                    number_of_cars: int, driving_distance_min: float,
                                    driving_distance_max: float, driving_speed: float,
                                    car_charging_min: float, car_charging_max: float,
                                    list_logs: List[Dict[str, Union[str, int, float]]]):
    for car_id in range(number_of_cars):
        charging_time = rng.uniform(car_charging_min, car_charging_max)
        driving_distance = rng.uniform(driving_distance_min, driving_distance_max)
        driving_time = 60 * driving_distance / driving_speed
        # use beta distribution below so most cars have
        #   lower priority (priority 1) than VIPs (priority 0)
        priority = np.round(rng.beta(5, 3)).astype(int)
        leave_time = rng.uniform(10., car_charging_max)
        env.process(electric_car_with_prio_leave_vip(
            env, charging_station, car_id, driving_time,
            charging_time, priority, leave_time,
            list_logs))
    print(f"At {env.now}, all cars have been added to the environment.")


def add_event_to_log(list_logs: List[Dict[str, Union[str, int, float]]],
                     event_time: float, car_id: int, car_prio: int,
                     drive_time: float, leave_time_left: float,
                     charge_time_left: float, event: str):
    list_logs.append({'Event time': event_time,
                      'Car ID': car_id, 
                      'Car priority': car_prio, 
                      'Drive time': drive_time,
                      'Available time left': leave_time_left,
                      'Charge time left': charge_time_left,
                      'Event': event})


def electric_car_with_prio_leave_vip(env: sp.Environment, charging_station: sp.PriorityResource,
                                     name: str, driving_time: float, charging_time: float,
                                     charging_station_priority: int, leave_time: float,
                                     list_logs: List[Dict[str, Union[str, int, float]]]):
    # setup placeholders for data
    charge_left = charging_time
    leave_left = leave_time
    timeout_event = None
    charging_timeout = None
    leave_timeout = None
    reenter_line = True
    
    # you can comment out all of the prints because we have a log now.
    # print(f"At {env.now:.2f}, Car {name} with t_drive {driving_time:.2f}, "
    #       f"t_charge {charging_time:.2f}, and t_leave {leave_time:.2f} has spawned.")
    add_event_to_log(list_logs, env.now, name, charging_station_priority, driving_time, 
                     leave_left, charge_left, "spawned")
    # print(f"At {env.now:.2f}, Car {name} is driving to the EV charging station.")
    add_event_to_log(list_logs, env.now, name, charging_station_priority, driving_time, 
                     leave_left, charge_left, "driving to charging station")
    yield env.timeout(driving_time)
    
    # while loop so we can re-insert this car into the queue automatically
    while reenter_line:
        with charging_station.request(priority=charging_station_priority) as charging_spot:
            wait_start = env.now
            leave_timeout = env.timeout(leave_left, value="leave_timeout")
            # print(f"At {env.now:.2f}, Car {name} with priority {charging_station_priority} "
            #       f"is waiting for an EV charging spot.")
            add_event_to_log(list_logs, env.now, name, charging_station_priority, driving_time, 
                             leave_left, charge_left, "waiting for spot")
            timeout_event = yield charging_spot | leave_timeout
            
            if leave_timeout in timeout_event:
                # print(f"At {env.now:.2f}, Car {name} gave up waiting in line...")
                add_event_to_log(list_logs, env.now, name, charging_station_priority, driving_time, 
                                 leave_left, charge_left, "gave up on charging station line")
                return

            wait_completed = env.now
            wait_duration = wait_completed - wait_start
            # print(f"At {env.now:.2f}, Car {name} with priority {charging_station_priority} "
            #       f"has acquired an EV charging spot after waiting {wait_duration:.2f}.")
            # NOTE: unfortunately, the wait duration isn't calculated until
            #       later so we include the wait duration as an event, we can
            #       extract the wait duration afterwards.
            add_event_to_log(list_logs, env.now, name, charging_station_priority, driving_time, 
                             leave_left, charge_left, f"acquired charging spot after waiting {wait_duration:.2f}")
            
            # We have to use a try-except-finally to handle the interrupt:
            try:
                charge_start = env.now
                charging_timeout = env.timeout(charge_left, value="charging_timeout")
                timeout_event = yield charging_timeout | leave_timeout
                reenter_line = False  # leave the line if we finished
            except sp.Interrupt as the_interrupt:
                interrupter = the_interrupt.cause.by
                # print(f"At {env.now:.2f}, Car {name} was interrupted by {interrupter}. "
                #       f"Getting back in line and waiting additional 10...")
                add_event_to_log(list_logs, env.now, name, charging_station_priority, driving_time, 
                                 leave_left, charge_left, "interrupted by vip")
                leave_left += 10.
            finally:
                # this lets us execute the computation in either case
                charge_duration = env.now - charge_start
                charge_left -= charge_duration
                leave_left -= charge_duration
    
    if charging_timeout in timeout_event:
        # print(f"At {env.now:.2f}, Car {name} has finished charging and left.")
        add_event_to_log(list_logs, env.now, name, charging_station_priority, driving_time, 
                         leave_left, charge_left, "finished charging and left")
    elif leave_timeout in timeout_event:
        # print(f"At {env.now:.2f}, Car {name} has to go somewhere and left with "
        #       f"{charge_left:.2f} charge required.")
        add_event_to_log(list_logs, env.now, name, charging_station_priority, driving_time, 
                         leave_left, charge_left, "left due to limit on available time")

In [ ]:
the_logs = []
the_rng = np.random.default_rng(RANDOM_SEED)
the_env = sp.Environment()
the_charging_station = sp.PreemptiveResource(the_env, capacity=CHARGING_SPOTS)
create_cars_with_prio_leave_vip(the_env, the_charging_station, the_rng, NUM_OF_CARS,
                                DISTANCE_MIN, DISTANCE_MAX, DRIVING_SPEED,
                                CHARGING_MIN, CHARGING_MAX, the_logs)
the_env.run()

In [ ]:
# check what's inside the logs
the_logs[:2]

In [ ]:
import pandas as pd

df_simulation = pd.DataFrame(the_logs)
df_simulation.head()

In [ ]:
df_waiting_rows = df_simulation[df_simulation["Event"].str.startswith("acquired charging spot after waiting")]
df_waiting_rows["Event"].str.split("acquired charging spot after waiting ")

Now, say that we want to find out the average amount of charge time left on all cars.  One of the ways we can get this is to group by the Car ID, then get the minimum charge time of each car (or last time event of each car), and then take the mean of that:

In [ ]:
mean_charge_time_starting = df_simulation.groupby(by="Car ID")["Charge time left"].max().mean()
std_charge_time_starting = df_simulation.groupby(by="Car ID")["Charge time left"].max().std()
mean_charge_time_left = df_simulation.groupby(by="Car ID")["Charge time left"].min().mean()
std_charge_time_left = df_simulation.groupby(by="Car ID")["Charge time left"].min().std()

print(f"The mean starting charge time left was {mean_charge_time_starting:.2f} +/- {std_charge_time_starting:.2f}.")
print(f"The mean final charge time left was {mean_charge_time_left:.2f} +/- {std_charge_time_left:.2f}.")

We see that for some reason, there are certain cars that did not get any charge time at all, pretty much.  If we were trying to optimize for a fair system, then we'd want to tweak our settings (perhaps get rid of the VIP provision!) until the standard deviation is much lower.  This is the power of discrete-event simulation -- we are able to run stochastic simulations that otherwise may be difficult to compute deterministically.

# Using SimPy for real-time ECE problems

Turns out that SimPy also has a RealtimeEnvironment, which lets you synchronize events with wall-clock time.

This means that we can perform the following simulations:
1. Hardware-in-the-loop testing (VERY useful in electronics, embedded systems, medical devices, networking, etc.)
2. Testing that requires human interaction (energy, sustainability, etc.)

**Let's discuss:** what might be an event-based simulation problem in your specialization